In [ ]:
! rm -rf ../output/*

In [ ]:
import os
os.chdir(os.getcwd() + "/..")

In [ ]:
from src.jobcontext import JobContext
from pyhocon import ConfigFactory

class Args:
    start = "2023-04-01"
    end = "2023-04-07"
    job = "SimpleJob"
    config = "resource/config.conf"

args = Args()

config = ConfigFactory.parse_file(args.config)
context = JobContext(config, args.job, args.start, args.end)
spark = context.create_spark_session()
context.spark = spark

In [ ]:
from src.jobs import SimpleJob

job = SimpleJob(context)
data = job.transform()

In [ ]:
pandas_df = data.toPandas()

In [ ]:
pandas_df

In [ ]:
spark.stop()